## For 3 categories

In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_excel("sample tweets.xlsx")

In [3]:
X=data['SentimentText']
Y=data['Sentiment']

In [4]:
with open('../glove.txt',encoding='mbcs') as f:
    doc=f.readlines()
    
glove_emb={
          }
for i in range(len(doc)):
    word   = doc[i].split(" ")[0]
    vector = np.array(doc[i].split(" ")[1:],dtype='float32')
    
    glove_emb[word]=vector

In [5]:
glove_emb['the']

array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
       -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
        2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
        1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
       -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
       -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
        4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
        7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
        1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01],
      dtype=float32)

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
rg=RegexpTokenizer("[a-zA-Z]+")
sw=stopwords.words("english")

In [7]:
unique={
    
}
weight=[]
k=1;
X_train=np.zeros((10000,45))
X_train=X_train.astype('int32')
for i in range(X.shape[0]):
    words=rg.tokenize(X[i])
    words=[w for w in words if w not in sw and w in glove_emb and len(w)>2]
    
    for j in range(min(len(words),45)):
        if words[j].lower() in unique:
            X_train[i][j]=unique[words[j].lower()]
            
        else:
            weight.append(glove_emb[words[j]])
            X_train[i][j]=k
            unique[words[j].lower()]=k
            k+=1

In [8]:
weight_matrix=[np.zeros((50,))]+weight
weight_matrix=np.array(weight_matrix)

In [9]:
weight_matrix.shape

(4185, 50)

In [10]:
from sklearn.preprocessing import LabelEncoder

In [11]:
le=LabelEncoder()

In [12]:
Y_encoded=le.fit_transform(Y)

In [13]:
from keras.utils import to_categorical
Y_train=to_categorical(Y_encoded)

Using TensorFlow backend.


In [14]:
Y_train

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

In [15]:
from keras.models import Sequential
from keras.layers import *

In [22]:
model=Sequential()
model.add(Embedding(4185,50, weights=[weight_matrix], input_length=45, trainable=False))
model.add(Reshape((45,50,1),input_shape=(45,50,)))
model.add(Conv2D(32,(3,3),activation="relu"))
model.add(MaxPool2D((2,2),strides=2))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(Flatten())
model.add(Dense(64,activation="relu"))
model.add(Dense(3,activation="softmax"))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 45, 50)            209250    
_________________________________________________________________
reshape_1 (Reshape)          (None, 45, 50, 1)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 43, 48, 32)        320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 21, 24, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 19, 22, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 9, 11, 64)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 7, 9, 64)         

In [23]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['acc'])

In [24]:
hist=model.fit(X_train,Y_train,validation_split=.2,epochs=15,batch_size=32)

Train on 8000 samples, validate on 2000 samples
Epoch 1/15
8000/8000 [==============================] - 19s 2ms/step - loss: 0.5978 - acc: 0.7534 - val_loss: 0.6812 - val_acc: 0.7025
Epoch 2/15
8000/8000 [==============================] - 18s 2ms/step - loss: 0.4534 - acc: 0.8170 - val_loss: 0.6199 - val_acc: 0.7485
Epoch 3/15
8000/8000 [==============================] - 19s 2ms/step - loss: 0.3754 - acc: 0.8559 - val_loss: 0.6145 - val_acc: 0.7470
Epoch 4/15
8000/8000 [==============================] - 18s 2ms/step - loss: 0.3256 - acc: 0.8748 - val_loss: 0.6443 - val_acc: 0.7470
Epoch 5/15
8000/8000 [==============================] - 19s 2ms/step - loss: 0.2806 - acc: 0.8892 - val_loss: 0.6418 - val_acc: 0.7660
Epoch 6/15
8000/8000 [==============================] - 18s 2ms/step - loss: 0.2542 - acc: 0.9026 - val_loss: 0.6632 - val_acc: 0.7585
Epoch 7/15
8000/8000 [==============================] - 19s 2ms/step - loss: 0.2236 - acc: 0.9131 - val_loss: 0.7176 - val_acc: 0.7695
Epoch 8

In [25]:
X_train.shape ,Y_train.shape

((10000, 45), (10000, 3))

In [26]:
Y_pred=model.predict_classes(X_train)

In [27]:
from sklearn.metrics import confusion_matrix

In [28]:
confusion_matrix(Y_encoded,Y_pred)

array([[6306,  268,   25],
       [ 408, 2465,   19],
       [ 111,   24,  374]], dtype=int64)

# for 2 categories
- removing entries for neutral

In [29]:
data=pd.read_excel("sample tweets.xlsx")

In [34]:
data2=data[data['Sentiment'] != 'Neutral'].reset_index(drop=True)

In [35]:
X2=data2['SentimentText']
Y2=data2['Sentiment']

In [36]:
unique={
    
}
weight=[]
k=1;
X_train=np.zeros((X2.shape[0],45))
X_train=X_train.astype('int32')
for i in range(X2.shape[0]):
    words=rg.tokenize(X2[i])
    words=[w for w in words if w not in sw and w in glove_emb and len(w)>2]
    
    for j in range(min(len(words),45)):
        if words[j].lower() in unique:
            X_train[i][j]=unique[words[j].lower()]
            
        else:
            weight.append(glove_emb[words[j]])
            X_train[i][j]=k
            unique[words[j].lower()]=k
            k+=1

In [37]:
weight_matrix=[np.zeros((50,))]+weight
weight_matrix=np.array(weight_matrix)

In [38]:
weight_matrix.shape

(3679, 50)

In [39]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
Y_train=le.fit_transform(Y2)

In [40]:
from keras.models import Sequential
from keras.layers import *

In [41]:
model=Sequential()
model.add(Embedding(3679,50, weights=[weight_matrix], input_length=45, trainable=False))
model.add(Reshape((45,50,1),input_shape=(45,50,)))
model.add(Conv2D(32,(3,3),activation="relu"))
model.add(MaxPool2D((2,2),strides=2))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(Flatten())
model.add(Dense(64,activation="relu"))
model.add(Dense(1,activation="sigmoid"))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 45, 50)            183950    
_________________________________________________________________
reshape_2 (Reshape)          (None, 45, 50, 1)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 43, 48, 32)        320       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 21, 24, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 19, 22, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 9, 11, 64)         0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 7, 9, 64)         

In [42]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['acc'])

In [ ]:
hist=model.fit(X_train,Y_train,validation_split=.2,epochs=15,batch_size=32)

Train on 5686 samples, validate on 1422 samples
Epoch 1/15
5686/5686 [==============================] - 13s 2ms/step - loss: 0.2343 - acc: 0.9275 - val_loss: 0.2880 - val_acc: 0.9100
Epoch 2/15
5686/5686 [==============================] - 13s 2ms/step - loss: 0.1921 - acc: 0.9330 - val_loss: 0.2662 - val_acc: 0.9100
Epoch 3/15
5686/5686 [==============================] - 13s 2ms/step - loss: 0.1669 - acc: 0.9372 - val_loss: 0.2491 - val_acc: 0.9191
Epoch 4/15
5686/5686 [==============================] - 13s 2ms/step - loss: 0.1403 - acc: 0.9553 - val_loss: 0.2592 - val_acc: 0.9156
Epoch 5/15
5686/5686 [==============================] - 14s 2ms/step - loss: 0.1196 - acc: 0.9592 - val_loss: 0.3163 - val_acc: 0.9170
Epoch 6/15
5686/5686 [==============================] - 14s 2ms/step - loss: 0.0987 - acc: 0.9678 - val_loss: 0.3231 - val_acc: 0.9226
Epoch 7/15
5686/5686 [==============================] - 14s 2ms/step - loss: 0.0795 - acc: 0.9724 - val_loss: 0.2939 - val_acc: 0.9072
Epoch 8

In [75]:
Y_pred=model.predict_classes(X_train)

In [76]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_train,Y_pred)

array([[6580,   19],
       [ 181,  328]], dtype=int64)